<a href="https://colab.research.google.com/github/sait1801/grid-test-cases/blob/main/carbon_prediction_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests

In [2]:
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('prod-last').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

print(rows)

# Convert to a DataFrame and render.
data = pd.DataFrame.from_records(rows)

[['Date', 'Hour', 'Total', 'Jeo', 'Dam', 'Solar', 'Flow', 'Wind'], ['16.02.2022', '0:00', '35720,65', '1319,58', '8649,08', '0', '2342,88', '1133,06'], ['16.02.2022', '1:00', '33564,98', '1319,3', '7280,35', '0', '2306,87', '1231,07'], ['16.02.2022', '2:00', '32468,18', '1323,4', '5852,76', '0', '2238,33', '1371,53'], ['16.02.2022', '3:00', '31699,17', '1323,12', '5076,98', '0', '2234,44', '1422,13'], ['16.02.2022', '4:00', '31554,16', '1325,2', '5092,94', '0', '2083,96', '1444,02'], ['16.02.2022', '5:00', '31901,53', '1313,09', '5540,07', ',07', '2046,8', '1472,11'], ['16.02.2022', '6:00', '33249,3', '1314,47', '6098,73', '6,32', '2008,2', '1533,4'], ['16.02.2022', '7:00', '35395,17', '1318,75', '6856,15', '37,13', '2066,59', '1628,16'], ['16.02.2022', '8:00', '39769,37', '1311,97', '7905,96', '164,36', '2454,83', '1630,32'], ['16.02.2022', '9:00', '41219,76', '1317,1', '7794,2', '346,56', '2627,63', '1556,91'], ['16.02.2022', '10:00', '40479,33', '1307,1', '6411,05', '487,89', '2606,

In [4]:
data.columns = ['Date'	,'Hour'	,'Consumption', 'Jeo', 'Dam', 'Wind' , 'Flow', 'Solar']
data= data.drop(0)

data.tail(5)

,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar
17516,14.02.2024,19:00,"39834,86","1235,36","9448,88",",01","3252,23","8208,96"
17517,14.02.2024,20:00,"42377,72","1233,66","9558,98",0,"3268,83","8513,98"
17518,14.02.2024,21:00,"39314,83","1212,17","9088,96",0,"3223,61","8330,17"
17519,14.02.2024,22:00,"39113,27","1245,47","9231,37",",01","2832,05","8102,75"
17520,14.02.2024,23:00,"36801,13","1166,73","8971,84",",01","2675,9","8027,69"


In [5]:
df=data.copy()
df.shape

(17520, 8)

##Adding WeekDay parameter to dataframe. The consumption may change daily due to weekin-weekend realtion

In [6]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['Weekday'] = 0

In [7]:
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i in range(len(df)):
  row = df.iloc[i]
  weekday = row['Date'].dayofweek
  df.loc[i, 'Weekday'] = weekday


In [8]:
df.tail(5)

,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday
17517,2024-02-14,20:00,"42377,72","1233,66","9558,98",0,"3268,83","8513,98",2.0
17518,2024-02-14,21:00,"39314,83","1212,17","9088,96",0,"3223,61","8330,17",2.0
17519,2024-02-14,22:00,"39113,27","1245,47","9231,37",",01","2832,05","8102,75",2.0
17520,2024-02-14,23:00,"36801,13","1166,73","8971,84",",01","2675,9","8027,69",0.0
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


##Changing the format of the energy to MGW

In [9]:
def convert_consumption(consumption):
  """Converts a string consumption to a float."""
  consumption = str(consumption).replace('.', '')
  consumption = str(consumption).replace(',', '.')
  consumption =float(consumption)
  consumption /=1000

  return consumption

In [10]:
for col in ['Consumption', 'Jeo', 'Dam', 'Wind' , 'Flow', 'Solar']:

  df[col] = df[col].apply(convert_consumption)

df.head(10)



,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday
1,2022-02-16,0:00,35.72065,1.31958,8.64908,0.00000,2.34288,1.13306,2.0
2,2022-02-16,1:00,33.56498,1.31930,7.28035,0.00000,2.30687,1.23107,2.0
3,2022-02-16,2:00,32.46818,1.32340,5.85276,0.00000,2.23833,1.37153,2.0
4,2022-02-16,3:00,31.69917,1.32312,5.07698,0.00000,2.23444,1.42213,2.0
5,2022-02-16,4:00,31.55416,1.32520,5.09294,0.00000,2.08396,1.44402,2.0
6,2022-02-16,5:00,31.90153,1.31309,5.54007,0.00007,2.04680,1.47211,2.0
7,2022-02-16,6:00,33.24930,1.31447,6.09873,0.00632,2.00820,1.53340,2.0
8,2022-02-16,7:00,35.39517,1.31875,6.85615,0.03713,2.06659,1.62816,2.0
9,2022-02-16,8:00,39.76937,1.31197,7.90596,0.16436,2.45483,1.63032,2.0
10,2022-02-16,9:00,41.21976,1.31710,7.79420,0.34656,2.62763,1.55691,2.0


In [11]:
df.tail(10)

,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday
17512,2024-02-14,15:00,41.30208,1.17140,7.95677,0.55428,2.91580,7.57184,2.0
17513,2024-02-14,16:00,41.00117,1.16134,8.90436,0.26308,2.97436,7.77065,2.0
17514,2024-02-14,17:00,43.54388,1.19535,9.77593,0.11394,3.07575,8.09842,2.0
17515,2024-02-14,18:00,42.45503,1.16692,9.66143,0.00710,3.27724,8.27601,2.0
17516,2024-02-14,19:00,39.83486,1.23536,9.44888,0.00001,3.25223,8.20896,2.0
17517,2024-02-14,20:00,42.37772,1.23366,9.55898,0.00000,3.26883,8.51398,2.0
17518,2024-02-14,21:00,39.31483,1.21217,9.08896,0.00000,3.22361,8.33017,2.0
17519,2024-02-14,22:00,39.11327,1.24547,9.23137,0.00001,2.83205,8.10275,2.0
17520,2024-02-14,23:00,36.80113,1.16673,8.97184,0.00001,2.67590,8.02769,0.0
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [12]:
# df['ren_rate']= (df['Consumption']-(df['Jeo']+df['Dam']+df['Wind']+df['Flow']+df['Solar']))
df['ren_rate']= ((df['Jeo']+df['Dam']+df['Wind']+df['Flow']+df['Solar'])/df['Consumption'])


##Add seasons and weekin-weekday parameters

In [13]:
df['Week_tag'] = 0
for i in range(len(df)):
  row = df.iloc[i]
  weekday = row['Weekday']
  if weekday in ['Sunday', 'Saturday']:
    df.loc[i, 'Week_tag'] = 1 #weekedn
  else:
    df.loc[i, 'Week_tag'] = 0 #weekin



In [14]:
# Assuming your DataFrame is named 'df'
df = df.dropna(subset=['Date'])

# Reset the index after removing the row
df.reset_index(drop=True, inplace=True)
df.tail(5)

,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday,ren_rate,Week_tag
17515,2024-02-14,19:00,39.83486,1.23536,9.44888,0.00001,3.25223,8.20896,2.0,0.555931,0
17516,2024-02-14,20:00,42.37772,1.23366,9.55898,0.00000,3.26883,8.51398,2.0,0.532720,0
17517,2024-02-14,21:00,39.31483,1.21217,9.08896,0.00000,3.22361,8.33017,2.0,0.555895,0
17518,2024-02-14,22:00,39.11327,1.24547,9.23137,0.00001,2.83205,8.10275,2.0,0.547427,0
17519,2024-02-14,23:00,36.80113,1.16673,8.97184,0.00001,2.67590,8.02769,0.0,0.566346,0


##Previous Consumption Data Adding

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df_sample =df.copy()


def convert_hour(hour):
  hour = str(hour)
  """Converts a time string in H:MM format to HH:MM format."""
  if len(hour) == 4:
    hour = '0' + hour[0]
  return hour

df_sample['Hour'] = df_sample['Hour'].apply(convert_hour)
print(df_sample['Hour'])
df_sample['Hour'] = df_sample['Hour'].str[:2].astype('int')

for i in range(1, 6):
    df_sample['previous-{}'.format(i)] = df_sample['ren_rate'].shift(i)
for i in range(1, 6):
    df_sample['previous-{}'.format(i)].fillna(df_sample['ren_rate'], inplace=True)


0           00
1           01
2           02
3           03
4           04
         ...  
17515    19:00
17516    20:00
17517    21:00
17518    22:00
17519    23:00
Name: Hour, Length: 17520, dtype: object


In [16]:
df_sample.head(6)


,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday,ren_rate,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
0,2022-02-16,0,35.72065,1.31958,8.64908,0.00000,2.34288,1.13306,2.0,0.376382,0,0.376382,0.376382,0.376382,0.376382,0.376382
1,2022-02-16,1,33.56498,1.31930,7.28035,0.00000,2.30687,1.23107,2.0,0.361615,0,0.376382,0.361615,0.361615,0.361615,0.361615
2,2022-02-16,2,32.46818,1.32340,5.85276,0.00000,2.23833,1.37153,2.0,0.332203,0,0.361615,0.376382,0.332203,0.332203,0.332203
3,2022-02-16,3,31.69917,1.32312,5.07698,0.00000,2.23444,1.42213,2.0,0.317253,0,0.332203,0.361615,0.376382,0.317253,0.317253
4,2022-02-16,4,31.55416,1.32520,5.09294,0.00000,2.08396,1.44402,2.0,0.315208,0,0.317253,0.332203,0.361615,0.376382,0.315208
5,2022-02-16,5,31.90153,1.31309,5.54007,0.00007,2.04680,1.47211,2.0,0.325130,0,0.315208,0.317253,0.332203,0.361615,0.376382


In [17]:
df_sample= df_sample.iloc[5:,:]
df_sample.head(10)

,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday,ren_rate,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
5,2022-02-16,5,31.90153,1.31309,5.54007,0.00007,2.04680,1.47211,2.0,0.325130,0,0.315208,0.317253,0.332203,0.361615,0.376382
6,2022-02-16,6,33.24930,1.31447,6.09873,0.00632,2.00820,1.53340,2.0,0.329665,0,0.325130,0.315208,0.317253,0.332203,0.361615
7,2022-02-16,7,35.39517,1.31875,6.85615,0.03713,2.06659,1.62816,2.0,0.336396,0,0.329665,0.325130,0.315208,0.317253,0.332203
8,2022-02-16,8,39.76937,1.31197,7.90596,0.16436,2.45483,1.63032,2.0,0.338639,0,0.336396,0.329665,0.325130,0.315208,0.317253
9,2022-02-16,9,41.21976,1.31710,7.79420,0.34656,2.62763,1.55691,2.0,0.330967,0,0.338639,0.336396,0.329665,0.325130,0.315208
10,2022-02-16,10,40.47933,1.30710,6.41105,0.48789,2.60648,1.24479,2.0,0.297863,0,0.330967,0.338639,0.336396,0.329665,0.325130
11,2022-02-16,11,39.80205,1.27625,6.12405,0.61348,2.55842,0.98433,2.0,0.290350,0,0.297863,0.330967,0.338639,0.336396,0.329665
12,2022-02-16,12,37.66125,1.25598,4.75428,0.70602,2.39563,0.90365,2.0,0.265938,0,0.290350,0.297863,0.330967,0.338639,0.336396
13,2022-02-16,13,37.53772,1.24435,4.31256,0.76038,2.35890,0.78229,2.0,0.251973,0,0.265938,0.290350,0.297863,0.330967,0.338639
14,2022-02-16,14,38.25473,1.24200,4.74139,0.73354,2.41605,0.82959,2.0,0.260427,0,0.251973,0.265938,0.290350,0.297863,0.330967


In [18]:
df_sample.dtypes

Date           datetime64[ns]
Hour                    int64
Consumption           float64
Jeo                   float64
Dam                   float64
Wind                  float64
Flow                  float64
Solar                 float64
Weekday               float64
ren_rate              float64
Week_tag                int64
previous-1            float64
previous-2            float64
previous-3            float64
previous-4            float64
previous-5            float64
dtype: object

##Train DNN

In [19]:
df_sample.head(5)

,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday,ren_rate,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
5,2022-02-16,5,31.90153,1.31309,5.54007,0.00007,2.04680,1.47211,2.0,0.325130,0,0.315208,0.317253,0.332203,0.361615,0.376382
6,2022-02-16,6,33.24930,1.31447,6.09873,0.00632,2.00820,1.53340,2.0,0.329665,0,0.325130,0.315208,0.317253,0.332203,0.361615
7,2022-02-16,7,35.39517,1.31875,6.85615,0.03713,2.06659,1.62816,2.0,0.336396,0,0.329665,0.325130,0.315208,0.317253,0.332203
8,2022-02-16,8,39.76937,1.31197,7.90596,0.16436,2.45483,1.63032,2.0,0.338639,0,0.336396,0.329665,0.325130,0.315208,0.317253
9,2022-02-16,9,41.21976,1.31710,7.79420,0.34656,2.62763,1.55691,2.0,0.330967,0,0.338639,0.336396,0.329665,0.325130,0.315208


In [20]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler  # Adapt for your scaler


# Preprocess data (adjust based on your specific requirements)
X = df_sample.drop(columns=['Date', 'Jeo', 'Dam', 'Wind' , 'Flow', 'Solar','ren_rate'])  # Exclude target variable #The target will be hour or stg ?  'Date', 'Consumption', 'Hour'
scaler_x = StandardScaler()
scaler_y = StandardScaler()
print(X.head(5))
X_scaled = scaler_x.fit_transform(X)

# Reshape data for LSTM input
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # Assuming a single timestep

Y = df_sample[['ren_rate']]
Y_scaled = scaler_y.fit_transform(Y)

# Reshape data for LSTM input
Y_reshaped = Y_scaled.reshape((Y_scaled.shape[0], 1, Y_scaled.shape[1]))  # Assuming a single timestep

# Define LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_reshaped.shape[1], X_reshaped.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))  # Single unit for regression output


   Hour  Consumption  Weekday  Week_tag  previous-1  previous-2  previous-3  \
5     5     31.90153      2.0         0    0.315208    0.317253    0.332203   
6     6     33.24930      2.0         0    0.325130    0.315208    0.317253   
7     7     35.39517      2.0         0    0.329665    0.325130    0.315208   
8     8     39.76937      2.0         0    0.336396    0.329665    0.325130   
9     9     41.21976      2.0         0    0.338639    0.336396    0.329665   

   previous-4  previous-5  
5    0.361615    0.376382  
6    0.332203    0.361615  
7    0.317253    0.332203  
8    0.315208    0.317253  
9    0.325130    0.315208  


In [21]:
X.head(5)

,Hour,Consumption,Weekday,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
5,5,31.90153,2.0,0,0.315208,0.317253,0.332203,0.361615,0.376382
6,6,33.24930,2.0,0,0.325130,0.315208,0.317253,0.332203,0.361615
7,7,35.39517,2.0,0,0.329665,0.325130,0.315208,0.317253,0.332203
8,8,39.76937,2.0,0,0.336396,0.329665,0.325130,0.315208,0.317253
9,9,41.21976,2.0,0,0.338639,0.336396,0.329665,0.325130,0.315208


In [22]:
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

In [23]:
from tensorflow.keras.callbacks import EarlyStopping
# Split data into training and testing sets (add validation set if desired)
train_size = int(0.8 * len(X_reshaped))  # Adjust split ratio as needed
X_train, X_test = X_reshaped[:train_size], X_reshaped[train_size:]
y_train, y_test = Y_reshaped[:train_size], Y_reshaped[train_size:]


early_stopping = EarlyStopping(
    monitor='val_loss',  # Adjust based on your chosen metric
    patience=10,  # Adapt patience to your training requirements
    min_delta=0.001,  # Define the minimum improvement threshold
    restore_best_weights=True  # Choose whether to restore best weights
)


# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate performance on test set
predictions = model.predict(X_test)


Epoch 1/100
438/438 [==============================] - 10s 9ms/step - loss: 0.1098 - val_loss: 0.0365
Epoch 2/100
438/438 [==============================] - 2s 6ms/step - loss: 0.0178 - val_loss: 0.0275
Epoch 3/100
438/438 [==============================] - 2s 6ms/step - loss: 0.0152 - val_loss: 0.0274
Epoch 4/100
438/438 [==============================] - 2s 6ms/step - loss: 0.0146 - val_loss: 0.0280
Epoch 5/100
438/438 [==============================] - 3s 7ms/step - loss: 0.0143 - val_loss: 0.0273
Epoch 6/100
438/438 [==============================] - 3s 6ms/step - loss: 0.0141 - val_loss: 0.0276
Epoch 7/100
438/438 [==============================] - 2s 6ms/step - loss: 0.0142 - val_loss: 0.0282
Epoch 8/100
438/438 [==============================] - 2s 6ms/step - loss: 0.0139 - val_loss: 0.0279
Epoch 9/100
438/438 [==============================] - 2s 6ms/step - loss: 0.0139 - val_loss: 0.0288
Epoch 10/100
438/438 [==============================] - 3s 7ms/step - loss: 0.0137 - val_l

In [24]:
print(y_test.shape)
y_test = y_test.reshape((y_test.shape[0], 1))
print(predictions.shape)

mse = mean_squared_error(y_test, predictions)
print('MSE:', mse)

(3503, 1, 1)
(3503, 1)
MSE: 0.02750911110186517


In [25]:
y_test_original = scaler_y.inverse_transform(y_test)
predictions_original = scaler_y.inverse_transform(predictions)

In [26]:
# Create the DataFrame
df_kiyas = pd.DataFrame({'y_test': y_test_original.flatten(), 'predictions': predictions_original.flatten()})

In [27]:
df_kiyas.head(10)

,y_test,predictions
0,0.177512,0.178660
1,0.164024,0.171639
2,0.161323,0.165773
3,0.160573,0.165938
4,0.155995,0.167765
5,0.147736,0.165398
6,0.159133,0.159024
7,0.204468,0.172523
8,0.224878,0.215860
9,0.223466,0.235081


In [28]:
import numpy as np

In [29]:
print(np.max(np.abs(df_kiyas['predictions']- df_kiyas['y_test'])))

0.08890778024757018


In [30]:
import plotly.graph_objects as go

# Assuming 'df' is your DataFrame
fig = go.Figure()
columns = ['y_original', 'predict_original']
df_kiyas['y_original'] = y_test_original
df_kiyas['predict_original'] = predictions_original



In [31]:
# Add a trace for each column
for col in columns:
    fig.add_trace(go.Scatter(x=df_kiyas.index, y=df_kiyas[col], mode='lines', name=col))

fig.show()


In [32]:
df_sample.tail(5)

,Date,Hour,Consumption,Jeo,Dam,Wind,Flow,Solar,Weekday,ren_rate,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
17515,2024-02-14,19,39.83486,1.23536,9.44888,0.00001,3.25223,8.20896,2.0,0.555931,0,0.527351,0.511194,0.513980,0.488355,0.450799
17516,2024-02-14,20,42.37772,1.23366,9.55898,0.00000,3.26883,8.51398,2.0,0.532720,0,0.555931,0.527351,0.511194,0.513980,0.488355
17517,2024-02-14,21,39.31483,1.21217,9.08896,0.00000,3.22361,8.33017,2.0,0.555895,0,0.532720,0.555931,0.527351,0.511194,0.513980
17518,2024-02-14,22,39.11327,1.24547,9.23137,0.00001,2.83205,8.10275,2.0,0.547427,0,0.555895,0.532720,0.555931,0.527351,0.511194
17519,2024-02-14,23,36.80113,1.16673,8.97184,0.00001,2.67590,8.02769,0.0,0.566346,0,0.547427,0.555895,0.532720,0.555931,0.527351


##Manuel Testing

##IF Not Persuaded, Let's try with haredcoded example

The data is :


Ren = (total - fosil) /total



In [33]:
data = [[19,43.89,4,0,0.59,0.57,0.56,0.56,0.56],[20,42.9,4,0,0.56,0.59,0.57,0.56,0.56],[21,41.54,4,0,0.55,0.56,0.59,0.57,0.56], [22,40.4,4,0,0.56,0.55,0.56,0.59,0.57], [23,38.3,4,0,0.58,0.56,0.55,0.56,0.59],[0,36.44,4,0,0.59,0.58,0.56,0.55,0.56]]
df_hard = pd.DataFrame(data, columns=['Hour', 'Consumption', 'Weekday', 'Week_tag', 'previous-1', 'previous-2', 'previous-3', 'previous-4', 'previous-5'])
df_hard.head(5)

,Hour,Consumption,Weekday,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
0,19,43.89,4,0,0.59,0.57,0.56,0.56,0.56
1,20,42.90,4,0,0.56,0.59,0.57,0.56,0.56
2,21,41.54,4,0,0.55,0.56,0.59,0.57,0.56
3,22,40.40,4,0,0.56,0.55,0.56,0.59,0.57
4,23,38.30,4,0,0.58,0.56,0.55,0.56,0.59


In [34]:
print(df_hard.shape)
df_hard.iloc[[4],:].head(1)

(6, 9)


,Hour,Consumption,Weekday,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
4,23,38.3,4,0,0.58,0.56,0.55,0.56,0.59


In [35]:
df_hard.shape

(6, 9)

In [36]:
print(df_hard.shape)
df_hard_train = scaler_x.transform(df_hard)

df_hard_train = df_hard_train.reshape((df_hard_train.shape[0], 1, df_hard_train.shape[1]))

print(df_hard_train.shape)

(6, 9)
(6, 1, 9)


In [37]:

pred_test = model.predict(df_hard_train)
print(scaler_y.inverse_transform(pred_test))

1/1 [==============================] - 0s 19ms/step
[[0.58385545]
 [0.55940735]
 [0.5468048 ]
 [0.5554402 ]
 [0.5750912 ]
 [0.57619417]]


In [38]:
df_test_2 = pd.DataFrame()
df_test_2['index'] = df_hard['Hour']
df_test_2['preeds']=  scaler_y.inverse_transform(pred_test)

In [39]:
df_test_2.head(6)

,index,preeds
0,19,0.583855
1,20,0.559407
2,21,0.546805
3,22,0.555440
4,23,0.575091
5,0,0.576194


In [40]:
df_test_2['real_values'] = [0.56, 0.55, 0.56, 0.58, 0.59, 0.56]

In [41]:
fig = go.Figure()

# Line plot example
fig.add_trace(go.Scatter(
    x=df_test_2.index,  # Use index for x-axis if plotting by row number
    y=df_test_2['preeds'],
    mode='lines+markers',
    name='predictions'
))
fig.add_trace(go.Scatter(
    x=df_test_2.index,
    y=df_test_2['real_values'],
    mode='lines+markers',
    name='real values2'
))


fig.show()


In [42]:
print(np.max(np.abs(df_test_2['preeds']- df_test_2['real_values'])))

0.024559812545776327


##TRy  Meta Prophet

In [43]:
# # Python
# import pandas as pd
# from prophet import Prophet



In [44]:
# df_prophet = df_sample[['Datetime']]

In [45]:
# # Python
# m = Prophet()
# m.fit(df)

##New Test 16.02-19.02 date.

In [46]:
!wget 'https://docs.google.com/spreadsheets/d/1lBklowv5rzcsh24UGR_kYcc1BE83tsVlYU4QG4uO1as/export?format=csv' -O carbon_test.csv


--2024-02-19 22:14:27--  https://docs.google.com/spreadsheets/d/1lBklowv5rzcsh24UGR_kYcc1BE83tsVlYU4QG4uO1as/export?format=csv
Resolving docs.google.com (docs.google.com)... 173.194.194.102, 173.194.194.100, 173.194.194.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.194.102|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-10-48-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/g0fndcuj6o5tg7lmcuelg6po0g/1708380865000/107047402098649921461/*/1lBklowv5rzcsh24UGR_kYcc1BE83tsVlYU4QG4uO1as?format=csv [following]
--2024-02-19 22:14:27--  https://doc-10-48-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/g0fndcuj6o5tg7lmcuelg6po0g/1708380865000/107047402098649921461/*/1lBklowv5rzcsh24UGR_kYcc1BE83tsVlYU4QG4uO1as?format=csv
Resolving doc-10-48-sheets.googleusercontent.com (doc-10-48-sheets.googleusercontent.com)... 172.253.114.132, 2607:f8b0:4001:c22::84
Connecting to doc-10-48-sheets.googleuser

###Data Cleaning

In [72]:
# Read the dataframe
df_test_new = pd.read_csv('/content/carbon_test.csv')

# Convert 'Date' column to datetime format
df_test_new['Date'] = pd.to_datetime(df_test_new['Date'], dayfirst=True)

# Add 'Weekday' column
df_test_new['Weekday'] = df_test_new['Date'].dt.dayofweek
df_test_new['Consumption'] = df_test_new["Total"]


In [73]:
def convert_consumption(consumption):
  """Converts a string consumption to a float."""
  consumption = str(consumption).replace('.', '')
  consumption = str(consumption).replace(',', '.')
  consumption = float(consumption)
  consumption /= 1000
  return consumption

for col in ['Consumption', 'Jeo', 'Dam', 'Wind' , 'Flow', 'Solar']:
  df_test_new[col] = df_test_new[col].apply(convert_consumption)


In [74]:
df_test_new['ren_rate'] = ((df_test_new['Jeo'] + df_test_new['Dam'] + df_test_new['Wind'] + df_test_new['Flow'] + df_test_new['Solar']) / df_test_new['Consumption'])
df_test_new['Week_tag'] = df_test_new['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
df_test_new = df_test_new.dropna(subset=['Date'])
df_test_new.reset_index(drop=True, inplace=True)
df_test3 = df_test_new.copy()

def convert_hour(hour):
  """Converts a time string in H:MM format to HH:MM format."""
  hour = str(hour)
  if len(hour) == 4:
    hour = '0' + hour[0]
  return hour

df_test3['Hour'] = df_test3['Hour'].apply(convert_hour)
df_test3['Hour'] = df_test3['Hour'].str[:2].astype('int')

for i in range(1, 6):
    df_test3['previous-{}'.format(i)] = df_test3['ren_rate'].shift(i)
for i in range(1, 6):
    df_test3['previous-{}'.format(i)].fillna(df_test3['ren_rate'], inplace=True)


In [75]:
df_test3 = df_test3.dropna()
df_test3.tail(5)

,Date,Hour,Total,Jeo,Dam,Wind,Flow,Solar,Weekday,Consumption,ren_rate,Week_tag,previous-1,previous-2,previous-3,previous-4,previous-5
79,2024-02-19,7,"33162,25",1.26013,8.33863,4.00602,2.39785,0.01035,0,33.16225,0.482868,0,0.479543,0.474159,0.500499,0.504660,0.489732
80,2024-02-19,8,"37441,54",1.13251,11.51724,3.87797,2.73479,0.13477,0,37.44154,0.518068,0,0.482868,0.479543,0.474159,0.500499,0.504660
81,2024-02-19,9,"38131,74",1.22599,11.85374,3.62945,2.73538,0.32926,0,38.13174,0.518566,0,0.518068,0.482868,0.479543,0.474159,0.500499
82,2024-02-19,10,"36854,07",1.10598,10.42382,3.47750,2.48905,0.44604,0,36.85407,0.486850,0,0.518566,0.518068,0.482868,0.479543,0.474159
83,2024-02-19,11,"231,84",0.00000,0.05320,0.00109,0.07434,0.00118,0,0.23184,0.559912,0,0.486850,0.518566,0.518068,0.482868,0.479543


##Make Predictions

In [76]:
df_test3 = df_test3[["Hour",	"Consumption"	,"Weekday",	"Week_tag"	,	"previous-1",	"previous-2"	,"previous-3"	,"previous-4",	"previous-5"]]

In [77]:

df_test3 = scaler_x.transform(df_test3)

# Reshape the data
df_test3 = df_test3.reshape((df_test3.shape[0], 1, df_test3.shape[1]))


In [78]:
# Assuming 'model' is your trained model
pred_test = model.predict(df_test3)

# Inverse transform the predictions
pred_test_inv = scaler_y.inverse_transform(pred_test)
# print(pred_test_inv)


3/3 [==============================] - 0s 4ms/step


In [79]:
pred_test_inv = pred_test_inv[1:]
print(pred_test_inv)

[[0.5191653 ]
 [0.5292532 ]
 [0.5315917 ]
 [0.52743423]
 [0.48673618]
 [0.4999625 ]
 [0.49198836]
 [0.48472267]
 [0.5509998 ]
 [0.5361126 ]
 [0.5175031 ]
 [0.5346926 ]
 [0.52471125]
 [0.53519326]
 [0.53298503]
 [0.5490138 ]
 [0.535584  ]
 [0.5533652 ]
 [0.5630432 ]
 [0.5676465 ]
 [0.5528058 ]
 [0.56308866]
 [0.5402771 ]
 [0.51219773]
 [0.50245434]
 [0.51176554]
 [0.5013986 ]
 [0.49898592]
 [0.5040755 ]
 [0.48749307]
 [0.47551295]
 [0.51489985]
 [0.54262143]
 [0.5384467 ]
 [0.5640083 ]
 [0.5419261 ]
 [0.5351134 ]
 [0.53482795]
 [0.535726  ]
 [0.52675325]
 [0.5328676 ]
 [0.5553216 ]
 [0.55226636]
 [0.543624  ]
 [0.52018124]
 [0.5430416 ]
 [0.5656246 ]
 [0.55030423]
 [0.5164906 ]
 [0.48235634]
 [0.4522937 ]
 [0.47576347]
 [0.466462  ]
 [0.4759026 ]
 [0.4362649 ]
 [0.44216678]
 [0.44731036]
 [0.49294803]
 [0.48017067]
 [0.53691566]
 [0.49768093]
 [0.5195093 ]
 [0.5153712 ]
 [0.5104066 ]
 [0.53129137]
 [0.52109414]
 [0.5236209 ]
 [0.54358983]
 [0.54710203]
 [0.56002533]
 [0.5526547 ]
 [0.49

In [80]:
# pred_test_inv = pred_test_inv + [0]*12
df_test_new = df_test_new.dropna()
df_test_new = df_test_new[:-1]
df_test_new['preds'] = pred_test_inv
df_test_new.head(5)

,Date,Hour,Total,Jeo,Dam,Wind,Flow,Solar,Weekday,Consumption,ren_rate,Week_tag,preds
0,2024-02-16,0:00,"34960,55",1.18600,8.88712,5.39291,3.08960,0.00001,4,34.96055,0.530759,0,0.519165
1,2024-02-16,1:00,"31508,83",1.19380,7.47275,5.31343,2.99625,0.00000,4,31.50883,0.538777,0,0.529253
2,2024-02-16,2:00,"30664,06",1.23708,7.25371,5.13101,2.86341,0.00001,4,30.66406,0.537607,0,0.531592
3,2024-02-16,3:00,"30055,08",1.20621,6.85112,5.04681,2.88097,0.00001,4,30.05508,0.531861,0,0.527434
4,2024-02-16,4:00,"31611,23",1.26407,6.56614,4.93557,2.85217,0.00000,4,31.61123,0.494063,0,0.486736


In [83]:
print(np.max(np.abs(df_test_new['preds'] - df_test_new['ren_rate'])))

0.022902145094153503


In [81]:
fig = go.Figure()

# Line plot example
fig.add_trace(go.Scatter(
    x=df_test_new.index,  # Use index for x-axis if plotting by row number
    y=df_test_new['preds'],
    mode='lines+markers',
    name='predictions'
))
fig.add_trace(go.Scatter(
    x=df_test_new.index,
    y=df_test_new['ren_rate'],
    mode='lines+markers',
    name='real values2'
))


fig.show()


In [82]:
df_test_new.head(5)

,Date,Hour,Total,Jeo,Dam,Wind,Flow,Solar,Weekday,Consumption,ren_rate,Week_tag,preds
0,2024-02-16,0:00,"34960,55",1.18600,8.88712,5.39291,3.08960,0.00001,4,34.96055,0.530759,0,0.519165
1,2024-02-16,1:00,"31508,83",1.19380,7.47275,5.31343,2.99625,0.00000,4,31.50883,0.538777,0,0.529253
2,2024-02-16,2:00,"30664,06",1.23708,7.25371,5.13101,2.86341,0.00001,4,30.66406,0.537607,0,0.531592
3,2024-02-16,3:00,"30055,08",1.20621,6.85112,5.04681,2.88097,0.00001,4,30.05508,0.531861,0,0.527434
4,2024-02-16,4:00,"31611,23",1.26407,6.56614,4.93557,2.85217,0.00000,4,31.61123,0.494063,0,0.486736
